In [ ]:
import librosa
import tensorflow as tf
import numpy as np


In [ ]:
file_name = "../Referencni_skladby/Imanbek & BYOR - Belly Dancer (Official Music Video).wav"

In [ ]:
y, sr = librosa.load(file_name,sr=22050, mono=True, duration=25)

In [ ]:
def get_melspec(y, sr):
    n_mels = 128
    hop_length = 512
    n_fft = 1024
    #calculate the melspectogram of the provided audio wave
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    
    return S

In [ ]:
mel = get_melspec(y,sr)

In [ ]:
vstupni_tensor = tf.constant(mel)

In [ ]:
vstupni_tensor_reshaped = tf.expand_dims(vstupni_tensor, axis=0)  # Přidává "batch size" dimenzi
vstupni_tensor_reshaped = tf.expand_dims(vstupni_tensor_reshaped, axis=-1)  # Přidává "channels" dimenzi

In [ ]:
saved_model_dir = 'model_muj_savedmodel_googleColab_trained/'

# Načtení modelu
model = tf.saved_model.load(saved_model_dir)
model_infer = model.signatures["serving_default"]

In [ ]:
vysledek = model_infer(vstupni_tensor_reshaped)
vysledek

In [ ]:
predictions_tensor = vysledek['dense_3']

In [ ]:
predictions = predictions_tensor.numpy()

In [ ]:
genres = {0: 'blues', 1 : 'classical', 2: 'country', 3 : 'disco', 4 : 'hiphop',  5 : 'jazz' , 6 : 'metal', 7 : 'pop', 8 : 'reggae', 9 : 'rock'}

In [ ]:
labels_list = [genres[i] for i in range(10)]

In [ ]:
# Vytvoření pole pro labely
labels_array = np.array(labels_list).reshape(-1, 1)  # Přeformátování na tvar (10, 1)

# Přeformátování výstupních dat, pokud je třeba
predictions = predictions.reshape(-1, 1)  # Ujistěte se, že má tvar (10, 1)

# Sloučení labelů a výstupních dat
final_array = np.hstack((labels_array, predictions))  # Sloučení do pole (10, 2)

print(final_array)